In [3]:
import seaborn as sns
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

PATH = ""
MODEL = "VisionTransformer_Base16"
FOLD = 4
EPOCH = 300


In [6]:
total_cmatrix = 0

for i in range(8,23):
    df = pd.read_csv(PATH+MODEL+f'\\1121_for_paper_results_datalist{i}_uncertainty_300epoch_iter50.csv')
    true_label = df['True Label']
    pred_label = df['Pred Label']
    total_cmatrix += confusion_matrix(true_label, pred_label)

total_cmatrix

array([[1605,   70,    5,    0,    0,    0,    0],
       [ 458,  380,   42,   20,    0,    0,    0],
       [  23,   89,  310,  283,    0,    0,    0],
       [   0,    2,   78, 1966,   54,    0,    0],
       [   0,    0,    0,  156,   89,   25,    0],
       [   0,    0,    0,   55,   21,  104,    0],
       [   0,    0,    0,   21,    0,   34,   20]], dtype=int64)

In [7]:
def acc(matrix,dataset_size):
        taikaku1 = sum(np.diag(matrix)) #対角成分
        taikaku2 = sum(np.diag(matrix, k=1)) + sum(np.diag(matrix, k=-1)) #対角成分の両サイド
        other1 = dataset_size-taikaku1#normal
        other2 = dataset_size-taikaku1-taikaku2 #1 neighbor
        # print(taikaku1,taikaku2)
        print(f'taikaku1:{taikaku1}, taikaku2:{taikaku2}, taikaku_total:{taikaku1+taikaku2}')
        print(f'dataset_size:{dataset_size}, other1:{other1}, other2:{other2}')
        return taikaku1/dataset_size,(taikaku1+taikaku2)/dataset_size,taikaku1,taikaku2


normal_acc, neighbor_acc, taikaku1,taikaku2 = acc(total_cmatrix,np.sum(total_cmatrix))
print()

print(f'normal_acc:{normal_acc}, neighbor_acc:{neighbor_acc}')
print(f'taikaku1:{taikaku1}, taikaku2:{taikaku2}')

taikaku1:4474, taikaku2:1310, taikaku_total:5784
dataset_size:5910, other1:1436, other2:126

normal_acc:0.7570219966159052, neighbor_acc:0.9786802030456853
taikaku1:4474, taikaku2:1310


In [ ]:
sns.set(font_scale = 2.5)
cm = pd.DataFrame(data=total_cmatrix, 
                  index=['Crowe 1, KL 1',
                         'Crowe 1, KL 2',
                         'Crowe 1, KL 3',
                         'Crowe 1, KL 4',
                         'Crowe 2, KL 4',
                         'Crowe 3, KL 4',
                         'Crowe 4, KL 4'], 
                  columns=['Crowe 1, KL 1',
                           'Crowe 1, KL 2',
                           'Crowe 1, KL 3',
                           'Crowe 1, KL 4',
                           'Crowe 2, KL 4',
                           'Crowe 3, KL 4',
                           'Crowe 4, KL 4'])

plt.figure(figsize=(15, 15))

#944枚使ったときにいい感じのグラデになるやつ#sns.heatmap(cm, 
#                                                    square=True, 
#                                                    cbar=True, 
#                                                    annot=True,
#                                                    cmap='Blues',
#                                                    fmt='d',
#                                                    vmax=200,
#                                                    vmin=0,
#                                                    center=120)

sns.heatmap(cm,
            square=True, 
            cbar=True,
            annot=True, 
            cmap='Blues',
            fmt='d',
            vmax=1000,
            vmin=0,
            center=500,
            annot_kws={"size":20})

#sns.heatmap(cm, square=True, cbar=True, annot=True, cmap='GnBu',fmt='d',vmax=40, vmin=-10, center=0)
#sns.heatmap(cm, square=True, cbar=True, annot=True, cmap='BuPu',fmt='d',vmax=40, vmin=-10, center=0)

plt.title(f'{MODEL}   {FOLD}Fold  {EPOCH}Epoch\nExact Class Acc: {normal_acc:.4f}'
          f'({taikaku1}/{np.sum(total_cmatrix)})\n1-Neighbor Class Acc: {neighbor_acc:.4f}  ({taikaku1+taikaku2}/{np.sum(total_cmatrix)})')
#plt.title(f'Confusion Matrix')
plt.yticks(rotation=0)
plt.xticks(rotation=45)
plt.xlabel("Predicted Label", fontsize=30, rotation=0)
plt.ylabel("True Label", fontsize=30)
plt.subplots_adjust(left=0.1, right=0.95, bottom=0.1, top=0.95)